In [1]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [45]:
def gpt_model_parameters():
    model = "gpt-3.5-turbo"
    temperature = 0.2
    max_tokens = 2048
    return model, temperature, max_tokens

In [41]:
base_message = [{"role": "system",
             "content": """The reviewer will enter the review they are writing.
This sentence is an incomplete sentence and all korean text.
A reviewer is korean, so you have to answer with korean.
If You have to figure out the context of the sentence and complete the rest of the sentence.
If the context requires a negative sentence, you must complete one negative sentence.
Conversely, if the context is positive, you must complete one positive sentence.
If you can't decide whether it's positive or negative, you need to complete two sentences, one positive and one negative.
You have to read the reviewer's review and write it by following the tone of the reviewer. 

This is an example.
"input" : "조식과 석식을 호텔에서 먹었는데"
<format>
{
  "results" : [
    {"긍정적", "조식과 석식을 호텔에서 먹었는데 맛과 다양성 모두 훌륭했습니다."}
    {"부정적", "조식과 석식을 호텔에서 먹었는데 식사 퀄리티가 기대에 미치지 않았습니다."}
  ]
}
"""}]

In [42]:
def generate_openai_response(base_message, query):
    model, temperature, max_tokens = gpt_model_parameters()
    # prompt message
    messages = base_message + [{"role": "user", "content": query}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature,
        max_tokens = max_tokens
    )
    return response["choices"][0]["message"]["content"]

In [10]:
def lambda_handler(event, context):
    try:
        body = json.loads(event.get('body', '{}'))
        message = body.get('message', '')

        if not message:
            raise ValueError('message is empty')

        messages.append({"role": "user", "content": message})
        response = generate_openai_response(message)
        response_body = {
            'message': response
        }
        return {
            'statusCode': 200,
            'body': json.dumps(response_body)
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': str(e)
        }

In [46]:
generate_openai_response(base_message, "바다가 보이는 60층에 묵었는데")

'<format>\n{\n  "results" : [\n    {"긍정적", "바다가 보이는 60층에 묵었는데 풍경이 너무 아름다웠습니다."},\n    {"부정적", "바다가 보이는 60층에 묵었는데 풍경이 기대에 못 미쳤습니다."}\n  ]\n}'

In [3]:
!pip install pylint